In [ ]:
from bindMods import SummaryTable
test = SummaryTable(r"E:\pharmacodynamics\sample data\summary table")

In [ ]:
test.make_binding_summary_tables()

In [ ]:
test.make_function_summary_tables()

In [ ]:
test.summary